<a href="https://colab.research.google.com/github/Colonna17/Collision_Avoidance-Sapienza_Vision_and_Perception/blob/main/CCD_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import torch
import torch.nn as nn
from torch.optim import Adam
#from Seq2Seq import Seq2Seq
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')



In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

In [ ]:
!nvidia-smi

Thu Dec 15 12:04:25 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    54W / 400W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!git clone --recurse-submodules https://github.com/Colonna17/Collision_Avoidance-Sapienza_Vision_and_Perception.git

%cd Collision_Avoidance-Sapienza_Vision_and_Perception
%pip install -qr external/yolov5/requirements.txt 
%pip install -qr requirements.txt #install dependencies

Cloning into 'Collision_Avoidance-Sapienza_Vision_and_Perception'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (142/142), done.
remote: Compressing objects: 100% (107/107), done.
remote: Total 306 (delta 46), reused 120 (delta 26), pack-reused 164
Receiving objects: 100% (306/306), 293.83 MiB | 61.47 MiB/s, done.
Resolving deltas: 100% (92/92), done.
Submodule 'Yolov5_StrongSORT_OSNet' (https://github.com/mikel-brostrom/Yolov5_StrongSORT_OSNet.git) registered for path 'external/Yolov5_StrongSORT_OSNet'
Submodule 'yolov5' (https://github.com/ultralytics/yolov5.git) registered for path 'external/yolov5'
Cloning into '/content/Collision_Avoidance-Sapienza_Vision_and_Perception/external/Yolov5_StrongSORT_OSNet'...
remote: Enumerating objects: 3356, done.        
remote: Counting objects: 100% (246/246), done.        
remote: Compressing objects: 100% (121/121), done.        
remote: Total 3356 (delta 147), reused 186 (delta 122), pack-reused 3110        
Receiv

In [ ]:
%cd /content/drive/MyDrive/Vision and Perception Project/Car Crash Dataset

/content/drive/.shortcut-targets-by-id/1pLOgUcJNt9NBV6ziPglNJ1kPKNX3b0It/Vision and Perception Project/Car Crash Dataset


In [ ]:
import os, cv2
import numpy as np
from google.colab.patches import cv2_imshow

def read_annotations_file(anno_file):
    assert os.path.exists(anno_file), "Annotation file does not exist!" + anno_file
    result = []
    with open(anno_file, 'r') as f:
        for line in f.readlines():
            items = {}
            items['vid'] = line.strip().split(',[')[0]
            labels = line.strip().split(',[')[1].split('],')[0]
            items['label'] = [int(val) for val in labels.split(',')]
            others = line.strip().split(',[')[1].split('],')[1].split(',')
            items['startframe'], items['vid_ytb'], items['lighting'], items['weather'], items['ego_involve'] = others
            result.append(items)
    f.close()
    return result


def get_video_frames(video_file, topN=50):
    # get the video data
    cap = cv2.VideoCapture(video_file)
    ret, frame = cap.read()
    video_data = []
    while (ret):
        video_data.append(frame)
        ret, frame = cap.read()
    #print("original # frames: %d"%(len(video_data)))
    assert len(video_data) >= topN
    video_data = video_data[:topN]
    return video_data

In [ ]:
tfs = transforms.Compose([
            transforms.Resize((720,1280)),
            transforms.ToTensor(),
            # normalize,
        ])

#Create the dataset 

In [ ]:
import numpy as np
videos_directory = "/content/drive/MyDrive/Vision and Perception Project/Car Crash Dataset/videos/Crash-1500/"
annotations_file = "/content/drive/MyDrive/Vision and Perception Project/Car Crash Dataset/videos/Crash-1500.txt"
#annotations_data = read_annotation_file(annotations_file)


# for anno in annotations_data:
#     video_file = os.path.join(videos_directory, anno['vid'] + ".mp4")
#     assert os.path.exists(video_file), "video file does not exist!" + video_file
#     # read frames
#     frames = get_video_frames(video_file, topN=50)
#     labels = anno['label']
#     #dataset = [dataset,frames]
#     # print("file: %s, # frames: %d, # labels: %d."%(video_file, len(frames), len(labels)))
#     # print(len(labels))
#     # for idx, im in enumerate(frames):
#     #     if labels[idx] == 1:
#     #         cv2.putText(im, 'Accident', (int(im.shape[1] / 2)-60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
#     #     else:
#     #         cv2.putText(im, 'Normal', (int(im.shape[1] / 2)-60, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 255), 2, cv2.LINE_AA)
#     #     cv2_imshow( im)
    #     cv2.waitKey(100)

In [ ]:
class CCDVideos(Dataset):
    def __init__(self, videos_directory, annotations_path):
        super().__init__()
        self.videos_paths = [ videos_directory + video_name for video_name in os.listdir(videos_directory) ]
        self.videos_paths.sort()
        self.annotations = read_annotations_file(annotations_path)

    def __getitem__(self, idx):
        video_frames = get_video_frames(self.videos_paths[idx])
        video_annotations = self.annotations[idx]
        return {'frames': video_frames, 'annotations': video_annotations}

    def __len__(self):
        return len(self.videos_paths)

    def read_annotations_file(annotations_path):
        assert os.path.exists(annotations_path), "Annotation file does not exist!" + annotations_path
        result = []
        with open(annotations_path, 'r') as f:
            for line in f.readlines():
                items = {}
                items['vid'] = line.strip().split(',[')[0]
                labels = line.strip().split(',[')[1].split('],')[0]
                items['label'] = [int(val) for val in labels.split(',')]
                others = line.strip().split(',[')[1].split('],')[1].split(',')
                items['startframe'], items['vid_ytb'], items['lighting'], items['weather'], items['ego_involve'] = others
                result.append(items)
        f.close()
        return result

# X: A tensor of size (B =batch_size, T =num_channels, C =seq_len, H = height, W = width)

    def get_video_frames(video_file, topN=50):
        # get the video data
        cap = cv2.VideoCapture(video_file)
        ret, frame = cap.read()
        
        video_data = []
        while (ret):
            video_data.append(frame)
            ret, frame = cap.read()
        #print("original # frames: %d"%(len(video_data)))
        assert len(video_data) >= topN
        video_data = video_data[:topN]
        return video_data
        # num_channels = video_data[0].shape[2]
        # seq_len = len(video_data)
        # height = video_data[0].shape[0]
        # width = video_data[0].shape[1]
        # video_data_np_array = np.zeros(num_channels, seq_len, height, width)
        # for i,video in  enumerate(video_data):
        #     video_data_np_array[:,i,:,:] = video
        # return video_data_np_array

In [ ]:
dir = '/content/drive/MyDrive/Vision and Perception Project/Car Crash Dataset/videos/Test-Crash/'
anno = '/content/drive/MyDrive/Vision and Perception Project/Car Crash Dataset/videos/test_annontations.txt'
dset = CCDVideos(dir ,anno)
loader = DataLoader(dset, batch_size=1, num_workers=0)

In [ ]:
for i,batch in enumerate(loader):
    print(batch['frames'][0].size())

torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])
torch.Size([1, 720, 1280, 3])


In [ ]:
import torch.nn as nn
import torch


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Original ConvLSTM cell as proposed by Shi et al.
class ConvLSTMCell(nn.Module):

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):
        """
        Initialize ConvLSTM cell.
        Parameters
        ----------
        in_channels: int
            Number of channels of input tensor.
        out_channels: int
            Number of channels of hidden state.
        kernel_size: (int, int)
            Size of the convolutional kernel.
        """

        super(ConvLSTMCell, self).__init__()  

        if activation == "tanh":
            self.activation = torch.tanh 
        elif activation == "relu":
            self.activation = torch.relu
        
        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        self.conv = nn.Conv2d(in_channels=in_channels + out_channels, 
                              out_channels=4 * out_channels, 
                              kernel_size=kernel_size, 
                              padding=padding)           

        # Initialize weights for Hadamard Products
        self.W_ci = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_co = nn.Parameter(torch.Tensor(out_channels, *frame_size))
        self.W_cf = nn.Parameter(torch.Tensor(out_channels, *frame_size))

    def forward(self, X, H_prev, C_prev):
        """
        The following are all 3D tensors: first dimension is n° of channels/filters
        while second and third are Height and Width of a frame.
        i: input gate
        f: forget gate
        o: output gate
        C: cell state
        H: hidden state
        X: inputs


        W: weights matrices. They are a learnable set of kernels.

        """

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        conv_output = self.conv(torch.cat([X, H_prev], dim=1))
        print('conv_output.size = ',conv_output.size())

        # Idea adapted from https://github.com/ndrplz/ConvLSTM_pytorch
        i_conv, f_conv, C_conv, o_conv = torch.chunk(conv_output, chunks=4, dim=1)
        
        print('i_conv.size() =', i_conv.size())


        input_gate = torch.sigmoid(i_conv + self.W_ci * C_prev )
        forget_gate = torch.sigmoid(f_conv + self.W_cf * C_prev )

        # Current Cell output
        C = forget_gate*C_prev + input_gate * self.activation(C_conv)

        output_gate = torch.sigmoid(o_conv + self.W_co * C )

        # Current Hidden State
        H = output_gate * self.activation(C)

        return H, C


class ConvLSTM(nn.Module): 
    """
    Parameters:
        input_dim: Number of channels in input
        hidden_dim: Number of hidden channels
        kernel_size: Size of kernel in convolutions
        num_layers: Number of LSTM layers stacked on each other
        batch_first: Whether or not dimension 0 is the batch or not
        bias: Bias or no bias in Convolution
        return_all_layers: Return the list of computations for all layers
        Note: Will do same padding.
    Input:
       X: A tensor of size (B =batch_size, T =num_channels, C =seq_len, H = height, W = width)
    Output:
        A tuple of two lists of length num_layers (or length 1 if return_all_layers is False).
            0 - layer_output_list is the list of lists of length T of each output
            1 - last_state_list is the list of last states
                    each element of the list is a tuple (h, c) for hidden state and memory
    Example:
        >> x = torch.rand((32, 10, 64, 128, 128))
        >> convlstm = ConvLSTM(64, 16, 3, 1, True, True, False)
        >> _, last_states = convlstm(x)
        >> h = last_states[0][0]  # 0 for layer index, 0 for h index
    """

    def __init__(self, in_channels, out_channels, 
    kernel_size, padding, activation, frame_size):

        super(ConvLSTM, self).__init__()

        self.out_channels = out_channels

        # We will unroll this over time steps. Remember that convLSTMcell returns
        # H hidden state and C cell
        self.convLSTMcell = ConvLSTMCell(in_channels, out_channels, 
        kernel_size, padding, activation, frame_size)

    def forward(self, X):

        # X is a frame sequence (B =batch_size, T =num_channels, C =seq_len, H = height, W = width)
        # Get the dimensions
        batch_size, _, seq_len, height, width = X.size()
        # Initialize output

        output = torch.zeros(batch_size, self.out_channels, seq_len, 
        height, width, device=device)
        
        # Initialize Hidden State
        H = torch.zeros(batch_size, self.out_channels, 
        height, width, device=device)
        print('H.size= ', H.size())

        # Initialize Cell Input
        C = torch.zeros(batch_size,self.out_channels, 
        height, width, device=device)
        print('C.size() = ', C.size())

        # Unroll over time steps
        for time_step in range(seq_len):

            H, C = self.convLSTMcell(X[:,:,time_step], H, C)

            output[:,:,time_step] = H

        return output


class Seq2Seq(nn.Module):

    def __init__(self, num_channels, num_kernels, kernel_size, padding, 
    activation, frame_size, num_layers):

        super(Seq2Seq, self).__init__()

        self.sequential = nn.Sequential()

        # Add First layer (Different in_channels than the rest)
        self.sequential.add_module(
            "convlstm1", ConvLSTM(
                in_channels=num_channels, out_channels=num_kernels,
                kernel_size=kernel_size, padding=padding, 
                activation=activation, frame_size=frame_size)
        )

        self.sequential.add_module(
            "batchnorm1", nn.BatchNorm3d(num_features=num_kernels)
        ) 

        # Add rest of the layers
        for l in range(2, num_layers+1):

            self.sequential.add_module(
                f"convlstm{l}", ConvLSTM(
                    in_channels=num_kernels, out_channels=num_kernels,
                    kernel_size=kernel_size, padding=padding, 
                    activation=activation, frame_size=frame_size)
                )
                
            self.sequential.add_module(
                f"batchnorm{l}", nn.BatchNorm3d(num_features=num_kernels)
                ) 

        # Add Convolutional Layer to predict output frame
        self.conv = nn.Conv2d(
            in_channels=num_kernels, out_channels=num_channels,
            kernel_size=kernel_size, padding=padding)

    def forward(self, X):

        # Forward propagation through all the layers
        output = self.sequential(X)

        # Return only the last output frame
        output = self.conv(output[:,:,-1])
        outpu = nn.Sigmoid()(output)
        
        return output

#TRAINING
From here on I'm gonna use pytorch tutorial for train the network


In [ ]:
# The input video frames are grayscale, thus single channel
model = Seq2Seq(num_channels=3, num_kernels=64, 
kernel_size=(5, 5), padding=(2,2), activation="relu", 
frame_size=(720, 1280), num_layers=3).to(device)

optim = Adam(model.parameters(), lr=1e-4)

# Binary Cross Entropy, target pixel values either 0 or 1
criterion = nn.BCELoss(reduction='sum')


In [ ]:
num_epochs = 1
torch.cuda.empty_cache()
for epoch in range(1, num_epochs+1):
    
    train_loss = 0                                                 
    model.train()                                                  
    for i, batch in enumerate(loader):  
        input = batch['frames']
        X = torch.stack(input, dim = 0).cuda()
        #Reshape X to make compliant with the model
        X = X.permute(1,4,0,2,3)
        output = model(X) 
        target = batch['annotations']['label']                                    
        loss = criterion(output.flatten(), target.flatten())       
        loss.backward()                                            
        optim.step()                                               
        optim.zero_grad()                                           
        train_loss += loss.item()
        
                                       
    train_loss /= len(loader.dataset)                       

    # val_loss = 0                                                 
    # model.eval()                                                   
    # with torch.no_grad():                                          
    #     for input, target in val_loader:                          
    #         output = model(input)                                   
    #         loss = criterion(output.flatten(), target.flatten())   
    #         val_loss += loss.item()                                
    # val_loss /= len(val_loader.dataset)                            

    print("Epoch:{} Training Loss:{:.2f} Validation Loss:{:.2f}\n".format(
        epoch, train_loss, val_loss))

H.size=  torch.Size([1, 64, 720, 1280])
C.size() =  torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.Size([1, 256, 720, 1280])
i_conv.size() = torch.Size([1, 64, 720, 1280])
conv_output.size =  torch.

OutOfMemoryError: ignored

In [ ]:
G = torch.zeros((1,3,5,7,12))
seqlenn = 5
for timestepp in range(seqlenn):
    mario = G[:,:,timestepp]
    print(mario.size())
    break